In [1]:
#Importing required libraries

import imblearn
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier


In [2]:
# Copy train dataset into dataframe

train_data_file = '/Users/mackbookair/Desktop/Git/CapStone/CapstoneProject/TestData/allstate-purchase-prediction-challenge-datawargling/train.csv'
train_df = pd.read_csv(train_data_file)
train_df.head()

,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,...,C_previous,duration_previous,A,B,C,D,E,F,G,cost
0,10000000,1,0,0,08:35,IN,10001,2,0,2,...,1.0,2.0,1,0,2,2,1,2,2,633
1,10000000,2,0,0,08:38,IN,10001,2,0,2,...,1.0,2.0,1,0,2,2,1,2,1,630
2,10000000,3,0,0,08:38,IN,10001,2,0,2,...,1.0,2.0,1,0,2,2,1,2,1,630
3,10000000,4,0,0,08:39,IN,10001,2,0,2,...,1.0,2.0,1,0,2,2,1,2,1,630
4,10000000,5,0,0,11:55,IN,10001,2,0,2,...,1.0,2.0,1,0,2,2,1,2,1,630


In [3]:
train_df.describe()

,customer_ID,shopping_pt,record_type,day,location,group_size,homeowner,car_age,risk_factor,age_oldest,...,C_previous,duration_previous,A,B,C,D,E,F,G,cost
count,6.652490e+05,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,424831.000000,665249.000000,...,646538.000000,646538.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000
mean,1.007655e+07,4.219966,0.145824,1.969429,12271.543022,1.234784,0.536229,8.139437,2.563996,44.992403,...,2.444718,6.003774,0.927546,0.454236,2.275265,2.454299,0.445193,1.162768,2.275120,635.785008
std,4.404978e+04,2.394369,0.352930,1.453470,1564.789415,0.461036,0.498686,5.764598,1.111584,17.403440,...,1.034596,4.680793,0.595221,0.497902,0.990798,0.753953,0.496987,0.943594,0.909567,45.993758
min,1.000000e+07,1.000000,0.000000,0.000000,10001.000000,1.000000,0.000000,0.000000,1.000000,18.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,260.000000
25%,1.003852e+07,2.000000,0.000000,1.000000,10936.000000,1.000000,0.000000,3.000000,2.000000,28.000000,...,1.000000,2.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,2.000000,605.000000
50%,1.007640e+07,4.000000,0.000000,2.000000,12027.000000,1.000000,1.000000,7.000000,3.000000,44.000000,...,3.000000,5.000000,1.000000,0.000000,2.000000,3.000000,0.000000,1.000000,2.000000,635.000000
75%,1.011470e+07,6.000000,0.000000,3.000000,13426.000000,1.000000,1.000000,12.000000,4.000000,60.000000,...,3.000000,9.000000,1.000000,1.000000,3.000000,3.000000,1.000000,2.000000,3.000000,665.000000
max,1.015272e+07,13.000000,1.000000,6.000000,16580.000000,4.000000,1.000000,85.000000,4.000000,75.000000,...,4.000000,15.000000,2.000000,1.000000,4.000000,3.000000,1.000000,3.000000,4.000000,922.000000


In [4]:
train_df.columns

Index(['customer_ID', 'shopping_pt', 'record_type', 'day', 'time', 'state',
       'location', 'group_size', 'homeowner', 'car_age', 'car_value',
       'risk_factor', 'age_oldest', 'age_youngest', 'married_couple',
       'C_previous', 'duration_previous', 'A', 'B', 'C', 'D', 'E', 'F', 'G',
       'cost'],
      dtype='object')

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665249 entries, 0 to 665248
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_ID        665249 non-null  int64  
 1   shopping_pt        665249 non-null  int64  
 2   record_type        665249 non-null  int64  
 3   day                665249 non-null  int64  
 4   time               665249 non-null  object 
 5   state              665249 non-null  object 
 6   location           665249 non-null  int64  
 7   group_size         665249 non-null  int64  
 8   homeowner          665249 non-null  int64  
 9   car_age            665249 non-null  int64  
 10  car_value          663718 non-null  object 
 11  risk_factor        424831 non-null  float64
 12  age_oldest         665249 non-null  int64  
 13  age_youngest       665249 non-null  int64  
 14  married_couple     665249 non-null  int64  
 15  C_previous         646538 non-null  float64
 16  du

Observations:

1. Total 665,249 records

2. 25 columns

3. Different data types -   
    a. time is identified as object      
    b. state and car_value are other columns which are also identified as objects   
    c. risk_factor, C_previous and duration_previous are identified as float  
    d. all other columns are identified as int  
    
4. Following columns have Null values -
    car_value, 
    risk_factor, 
    C_previous, 
    duration_previous
    
5. column name 'customer_ID' is inconsistent

6. Missing values:     
    a. 240,418 records does not have value for risk_factor, which approximates to 36% of total records  
    b. 1,531 records does not have value for car_value, which approximates to 0.23% of total records  
    c. 18,711 records does not have value for C_previous, which approximates to 2.81% of total records   
    d. 18,711 records does not have value for duration_previous, which approximates to 2.81% of total records

Lot of records does not have a value for risk_factor. 

In [6]:
# Chekcing for unique values in risk_factor
train_df.risk_factor.unique()

array([ 3.,  4., nan,  2.,  1.])

In [7]:
# Checking for number of records for each distinct value
train_df['risk_factor'].value_counts()

3.0    117571
4.0    110754
1.0     99476
2.0     97030
Name: risk_factor, dtype: int64

In [8]:
# Caliculating mean of the column
risk_factor_mean = round(train_df['risk_factor'].mean())
risk_factor_mean

3

In [9]:
# Replacing na values with the mean value 
train_df['risk_factor']=train_df['risk_factor'].fillna(round((train_df['risk_factor'].mean())))
train_df['risk_factor'].value_counts()

3.0    357989
4.0    110754
1.0     99476
2.0     97030
Name: risk_factor, dtype: int64

In [10]:
train_df.risk_factor.unique()

array([3., 4., 2., 1.])

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665249 entries, 0 to 665248
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_ID        665249 non-null  int64  
 1   shopping_pt        665249 non-null  int64  
 2   record_type        665249 non-null  int64  
 3   day                665249 non-null  int64  
 4   time               665249 non-null  object 
 5   state              665249 non-null  object 
 6   location           665249 non-null  int64  
 7   group_size         665249 non-null  int64  
 8   homeowner          665249 non-null  int64  
 9   car_age            665249 non-null  int64  
 10  car_value          663718 non-null  object 
 11  risk_factor        665249 non-null  float64
 12  age_oldest         665249 non-null  int64  
 13  age_youngest       665249 non-null  int64  
 14  married_couple     665249 non-null  int64  
 15  C_previous         646538 non-null  float64
 16  du

In [12]:
# Droping rest of records having nan values
train_df = train_df.dropna()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_ID        645648 non-null  int64  
 1   shopping_pt        645648 non-null  int64  
 2   record_type        645648 non-null  int64  
 3   day                645648 non-null  int64  
 4   time               645648 non-null  object 
 5   state              645648 non-null  object 
 6   location           645648 non-null  int64  
 7   group_size         645648 non-null  int64  
 8   homeowner          645648 non-null  int64  
 9   car_age            645648 non-null  int64  
 10  car_value          645648 non-null  object 
 11  risk_factor        645648 non-null  float64
 12  age_oldest         645648 non-null  int64  
 13  age_youngest       645648 non-null  int64  
 14  married_couple     645648 non-null  int64  
 15  C_previous         645648 non-null  float64
 16  du

We have 645,648 out of 665,249 total records, we droped around 3% of records which have nan value.

In [13]:
#Renaming column names - converting uppercase names to lower case
train_df.rename(columns={'customer_ID':'customer_id','C_previous':'c_previous','A':'a','B':'b','C':'c','D':'d','E':'e','F':'f','G':'g'},inplace = True)

train_df.columns

Index(['customer_id', 'shopping_pt', 'record_type', 'day', 'time', 'state',
       'location', 'group_size', 'homeowner', 'car_age', 'car_value',
       'risk_factor', 'age_oldest', 'age_youngest', 'married_couple',
       'c_previous', 'duration_previous', 'a', 'b', 'c', 'd', 'e', 'f', 'g',
       'cost'],
      dtype='object')

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_id        645648 non-null  int64  
 1   shopping_pt        645648 non-null  int64  
 2   record_type        645648 non-null  int64  
 3   day                645648 non-null  int64  
 4   time               645648 non-null  object 
 5   state              645648 non-null  object 
 6   location           645648 non-null  int64  
 7   group_size         645648 non-null  int64  
 8   homeowner          645648 non-null  int64  
 9   car_age            645648 non-null  int64  
 10  car_value          645648 non-null  object 
 11  risk_factor        645648 non-null  float64
 12  age_oldest         645648 non-null  int64  
 13  age_youngest       645648 non-null  int64  
 14  married_couple     645648 non-null  int64  
 15  c_previous         645648 non-null  float64
 16  du

In [15]:
train_df.time.unique()

array(['08:35', '08:38', '08:39', ..., '00:13', '00:16', '00:40'],
      dtype=object)

In [16]:
# dropping time column
train_df = train_df.drop('time',axis=1)
train_df.columns

Index(['customer_id', 'shopping_pt', 'record_type', 'day', 'state', 'location',
       'group_size', 'homeowner', 'car_age', 'car_value', 'risk_factor',
       'age_oldest', 'age_youngest', 'married_couple', 'c_previous',
       'duration_previous', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'cost'],
      dtype='object')

In [17]:
train_df.customer_id.unique()

array([10000000, 10000005, 10000007, ..., 10152721, 10152723, 10152724])

In [18]:
# dropping customer_id column
train_df = train_df.drop('customer_id',axis=1)
train_df.columns

Index(['shopping_pt', 'record_type', 'day', 'state', 'location', 'group_size',
       'homeowner', 'car_age', 'car_value', 'risk_factor', 'age_oldest',
       'age_youngest', 'married_couple', 'c_previous', 'duration_previous',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'cost'],
      dtype='object')

In [19]:
train_df.day.unique()

array([0, 3, 4, 2, 1, 5, 6])

In [20]:
train_df['day'].value_counts()

0    136441
1    132807
2    129654
4    119937
3    117781
5      8088
6       940
Name: day, dtype: int64

In [21]:
train_df['day'][train_df.record_type==0].value_counts()

0    118009
1    113738
2    110982
3     99980
4     99273
5      6672
6       821
Name: day, dtype: int64

In [22]:
# dropping day column
#train_df = train_df.drop('day',axis=1)
train_df.columns

Index(['shopping_pt', 'record_type', 'day', 'state', 'location', 'group_size',
       'homeowner', 'car_age', 'car_value', 'risk_factor', 'age_oldest',
       'age_youngest', 'married_couple', 'c_previous', 'duration_previous',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'cost'],
      dtype='object')

In [23]:
# dropping day column
#train_df = train_df.drop('shopping_pt',axis=1)
#train_df.columns

In [24]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   shopping_pt        645648 non-null  int64  
 1   record_type        645648 non-null  int64  
 2   day                645648 non-null  int64  
 3   state              645648 non-null  object 
 4   location           645648 non-null  int64  
 5   group_size         645648 non-null  int64  
 6   homeowner          645648 non-null  int64  
 7   car_age            645648 non-null  int64  
 8   car_value          645648 non-null  object 
 9   risk_factor        645648 non-null  float64
 10  age_oldest         645648 non-null  int64  
 11  age_youngest       645648 non-null  int64  
 12  married_couple     645648 non-null  int64  
 13  c_previous         645648 non-null  float64
 14  duration_previous  645648 non-null  float64
 15  a                  645648 non-null  int64  
 16  b 

Inspecting columns with type as float64

In [25]:
train_df.risk_factor.unique()

array([3., 4., 2., 1.])

In [26]:
# Convert risk_factor column to int type
train_df['risk_factor']=train_df['risk_factor'].astype('int')
train_df.risk_factor.unique()

array([3, 4, 2, 1])

In [27]:
train_df.c_previous.unique()

array([1., 3., 2., 4.])

In [28]:
# Convert c_previous column to int type
train_df['c_previous']=train_df['c_previous'].astype('int')
train_df.c_previous.unique()

array([1, 3, 2, 4])

In [29]:
 train_df.duration_previous.unique()

array([ 2., 13.,  4.,  3.,  9.,  0.,  1.,  8., 14.,  5., 15.,  6.,  7.,
       11., 10., 12.])

In [30]:
# Convert duration_previous column to int type
train_df['duration_previous']=train_df['duration_previous'].astype('int')
train_df.duration_previous.unique()

array([ 2, 13,  4,  3,  9,  0,  1,  8, 14,  5, 15,  6,  7, 11, 10, 12])

In [31]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   shopping_pt        645648 non-null  int64 
 1   record_type        645648 non-null  int64 
 2   day                645648 non-null  int64 
 3   state              645648 non-null  object
 4   location           645648 non-null  int64 
 5   group_size         645648 non-null  int64 
 6   homeowner          645648 non-null  int64 
 7   car_age            645648 non-null  int64 
 8   car_value          645648 non-null  object
 9   risk_factor        645648 non-null  int64 
 10  age_oldest         645648 non-null  int64 
 11  age_youngest       645648 non-null  int64 
 12  married_couple     645648 non-null  int64 
 13  c_previous         645648 non-null  int64 
 14  duration_previous  645648 non-null  int64 
 15  a                  645648 non-null  int64 
 16  b                  6

In [32]:
train_df.head()

,shopping_pt,record_type,day,state,location,group_size,homeowner,car_age,car_value,risk_factor,...,c_previous,duration_previous,a,b,c,d,e,f,g,cost
0,1,0,0,IN,10001,2,0,2,g,3,...,1,2,1,0,2,2,1,2,2,633
1,2,0,0,IN,10001,2,0,2,g,3,...,1,2,1,0,2,2,1,2,1,630
2,3,0,0,IN,10001,2,0,2,g,3,...,1,2,1,0,2,2,1,2,1,630
3,4,0,0,IN,10001,2,0,2,g,3,...,1,2,1,0,2,2,1,2,1,630
4,5,0,0,IN,10001,2,0,2,g,3,...,1,2,1,0,2,2,1,2,1,630


In [33]:
train_df.state.unique()

array(['IN', 'NY', 'PA', 'WV', 'MO', 'OH', 'OK', 'FL', 'OR', 'WA', 'KS',
       'NV', 'ID', 'CO', 'CT', 'AL', 'AR', 'NM', 'MS', 'MD', 'RI', 'ME',
       'TN', 'UT', 'WI', 'MT', 'KY', 'WY', 'NE', 'ND', 'DE', 'GA', 'NH',
       'IA', 'DC', 'SD'], dtype=object)

In [34]:
train_df['state'].value_counts()

FL    104988
NY     89589
PA     59049
OH     43417
MD     27957
WA     24537
IN     23891
CO     23260
AL     22972
CT     18847
TN     17537
KY     15589
NV     15092
MO     14759
OR     14207
UT     14167
OK     13526
MS     10258
AR     10223
WI      9954
GA      8484
NH      7312
NM      6676
ME      6370
ID      5873
RI      5711
KS      5490
WV      4989
IA      4436
DE      3996
DC      3271
MT      3110
NE      3087
ND      1454
WY      1133
SD       437
Name: state, dtype: int64

In [35]:
train_df['state'][train_df.record_type==1].value_counts()

FL    14599
NY    13574
PA     8900
OH     6641
MD     4218
IN     3639
AL     3578
WA     3561
CO     3516
CT     2813
TN     2635
KY     2348
MO     2193
OR     2138
NV     2119
UT     2069
OK     2018
AR     1561
MS     1557
WI     1544
GA     1254
NH     1091
ME     1002
NM      999
ID      885
RI      873
KS      858
WV      779
IA      686
DE      595
DC      502
NE      490
MT      461
ND      240
WY      166
SD       71
Name: state, dtype: int64

In [36]:
train_df.location.unique()

array([10001, 10006, 10008, ..., 16579, 16562, 16580])

In [37]:
train_df['location'].value_counts()

10083    1027
10213     974
11517     864
10348     860
10030     782
         ... 
15697       1
16166       1
16345       1
14702       1
16405       1
Name: location, Length: 6244, dtype: int64

In [38]:
train_df['group_size'].value_counts()

1    503170
2    133112
3      8686
4       680
Name: group_size, dtype: int64

In [39]:
train_df['group_size'][train_df.record_type==1].value_counts()

1    74924
2    19865
3     1274
4      110
Name: group_size, dtype: int64

In [40]:
train_df['homeowner'].value_counts()

1    348357
0    297291
Name: homeowner, dtype: int64

In [41]:
train_df['homeowner'][train_df.record_type==1].value_counts()

1    52620
0    43553
Name: homeowner, dtype: int64

In [42]:
train_df['car_age'].value_counts()

1     69635
2     49355
7     45681
6     43408
8     43217
      ...  
65        5
51        5
60        4
85        4
54        1
Name: car_age, Length: 67, dtype: int64

In [43]:
train_df['car_age'][train_df.record_type==1].value_counts()

1     10290
2      7292
7      6772
8      6424
6      6399
      ...  
57        1
58        1
65        1
60        1
85        1
Name: car_age, Length: 65, dtype: int64

In [44]:
train_df['car_value'].value_counts()

e    213495
f    172456
d    109861
g     95497
h     28224
c     20179
i      3520
b      1342
a      1074
Name: car_value, dtype: int64

In [45]:
train_df['car_value'][train_df.record_type==1].value_counts()

e    31884
f    25725
d    16252
g    14283
h     4126
c     3027
i      500
b      206
a      170
Name: car_value, dtype: int64

In [46]:
train_df['age_oldest'].value_counts()

75    44133
24    20875
23    20751
25    20617
22    18930
26    18121
27    15494
28    14751
21    13686
29    12771
30    12371
31    11636
32    11279
33    11079
43    11039
55    10842
42    10747
52    10686
34    10658
53    10490
46    10458
35    10385
56    10380
49    10308
54    10283
48    10268
58    10211
51    10130
50     9983
59     9908
45     9860
44     9839
41     9815
20     9700
36     9656
57     9615
47     9589
37     9532
66     9518
65     9455
63     9444
38     9440
60     9417
40     9321
61     9257
39     9192
62     9149
64     9076
70     8092
69     7458
67     7391
68     6819
71     6738
19     6210
72     6055
73     5724
74     5551
18     1465
Name: age_oldest, dtype: int64

In [47]:
train_df['age_youngest'].value_counts()

75    33482
23    23386
25    22779
24    22765
22    21464
26    19886
27    17154
21    16677
28    16383
29    14013
30    13268
20    12903
31    12625
32    12137
33    11663
34    11139
35    10821
55     9992
43     9858
42     9826
52     9715
19     9678
36     9624
48     9507
37     9495
46     9469
58     9454
53     9452
51     9382
49     9371
56     9349
38     9316
50     9263
54     9251
41     9223
40     9084
45     9039
44     8981
65     8936
57     8894
39     8834
60     8686
47     8674
59     8666
66     8661
62     8539
61     8490
63     8415
64     8353
70     7142
69     6870
68     6295
67     6075
71     6021
72     5148
73     5069
74     4931
18     4835
16     4380
17     2860
Name: age_youngest, dtype: int64

In [48]:
train_df['married_couple'].value_counts()

0    509473
1    136175
Name: married_couple, dtype: int64

In [49]:
train_df['c_previous'].value_counts()

3    270882
1    171605
2    109025
4     94136
Name: c_previous, dtype: int64

In [50]:
train_df['duration_previous'].value_counts()

1     81270
15    79825
2     79388
3     70682
4     57417
5     49332
6     45356
7     37754
8     30734
9     26235
0     24874
10    18092
11    12715
12    11282
13    10955
14     9737
Name: duration_previous, dtype: int64

In [51]:
train_df['a'].value_counts()

1    416650
0    135744
2     93254
Name: a, dtype: int64

In [52]:
train_df['b'].value_counts()

0    349542
1    296106
Name: b, dtype: int64

In [53]:
train_df['c'].value_counts()

3    266103
1    193609
2    129597
4     56339
Name: c, dtype: int64

In [54]:
train_df['d'].value_counts()

3    401183
2    145026
1     99439
Name: d, dtype: int64

In [55]:
train_df['e'].value_counts()

0    354557
1    291091
Name: e, dtype: int64

In [56]:
train_df['f'].value_counts()

2    249890
0    206957
1    155474
3     33327
Name: f, dtype: int64

In [57]:
train_df['g'].value_counts()

2    251112
3    189315
1    139280
4     65941
Name: g, dtype: int64

In [58]:
train_df['cost'].value_counts()

633    5901
637    5869
626    5824
638    5808
640    5801
       ... 
418       1
416       1
851       1
341       1
863       1
Name: cost, Length: 524, dtype: int64

In [59]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   shopping_pt        645648 non-null  int64 
 1   record_type        645648 non-null  int64 
 2   day                645648 non-null  int64 
 3   state              645648 non-null  object
 4   location           645648 non-null  int64 
 5   group_size         645648 non-null  int64 
 6   homeowner          645648 non-null  int64 
 7   car_age            645648 non-null  int64 
 8   car_value          645648 non-null  object
 9   risk_factor        645648 non-null  int64 
 10  age_oldest         645648 non-null  int64 
 11  age_youngest       645648 non-null  int64 
 12  married_couple     645648 non-null  int64 
 13  c_previous         645648 non-null  int64 
 14  duration_previous  645648 non-null  int64 
 15  a                  645648 non-null  int64 
 16  b                  6

Converting 'state','location','group_size','homeowner','car_value','risk_factor','married_couple','c_previous','a','b','c','d','e','f' and 'g' columns to category type

In [60]:
categorical_cols = ['state','location','group_size','homeowner','car_value','risk_factor','married_couple',
                    'c_previous','a','b','c','d','e','f','g'] 

In [61]:
for col in categorical_cols:
  train_df[col] = train_df[col].astype('category')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   shopping_pt        645648 non-null  int64   
 1   record_type        645648 non-null  int64   
 2   day                645648 non-null  int64   
 3   state              645648 non-null  category
 4   location           645648 non-null  category
 5   group_size         645648 non-null  category
 6   homeowner          645648 non-null  category
 7   car_age            645648 non-null  int64   
 8   car_value          645648 non-null  category
 9   risk_factor        645648 non-null  category
 10  age_oldest         645648 non-null  int64   
 11  age_youngest       645648 non-null  int64   
 12  married_couple     645648 non-null  category
 13  c_previous         645648 non-null  category
 14  duration_previous  645648 non-null  int64   
 15  a                  645648 non-null

In [62]:
# instantiate labelencoder object
le = LabelEncoder()
# apply le on categorical feature columns
train_df[categorical_cols] = train_df[categorical_cols].apply(lambda col: le.fit_transform(col))
train_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   shopping_pt        645648 non-null  int64
 1   record_type        645648 non-null  int64
 2   day                645648 non-null  int64
 3   state              645648 non-null  int64
 4   location           645648 non-null  int64
 5   group_size         645648 non-null  int64
 6   homeowner          645648 non-null  int64
 7   car_age            645648 non-null  int64
 8   car_value          645648 non-null  int64
 9   risk_factor        645648 non-null  int64
 10  age_oldest         645648 non-null  int64
 11  age_youngest       645648 non-null  int64
 12  married_couple     645648 non-null  int64
 13  c_previous         645648 non-null  int64
 14  duration_previous  645648 non-null  int64
 15  a                  645648 non-null  int64
 16  b                  645648 non-null  in

In [63]:
train_df['record_type'].value_counts()

0    549475
1     96173
Name: record_type, dtype: int64

There are 549,475 records with 0 record_type and only 96,173 records with 1. Which is about 14% of total records 645,648 records. 

Any model trained on this data set would have over sampling records for record_type as 1)

In [64]:
df_s = train_df
df_sample = df_s.sample(frac=0.1, replace=True)

In [65]:
df_sample.head()

,shopping_pt,record_type,day,state,location,group_size,homeowner,car_age,car_value,risk_factor,...,c_previous,duration_previous,a,b,c,d,e,f,g,cost
16051,5,0,4,30,476,0,1,15,5,2,...,0,9,1,1,0,0,1,2,1,673
163505,8,1,2,27,647,0,0,7,4,2,...,0,4,1,0,0,1,1,1,2,686
28919,9,1,3,10,1505,0,1,7,3,2,...,2,1,2,1,2,2,0,2,2,598
180126,9,1,1,27,1689,0,1,6,3,3,...,0,3,2,0,0,0,0,3,0,637
336471,6,0,1,32,4460,1,1,14,5,2,...,2,1,0,0,2,2,0,0,2,641


In [66]:
df_sample.info

<bound method DataFrame.info of         shopping_pt  record_type  day  state  location  group_size  homeowner  \
16051             5            0    4     30       476           0          1   
163505            8            1    2     27       647           0          0   
28919             9            1    3     10      1505           0          1   
180126            9            1    1     27      1689           0          1   
336471            6            0    1     32      4460           1          1   
...             ...          ...  ...    ...       ...         ...        ...   
98737             6            1    3     12      4842           0          1   
153853            2            0    2     23       252           0          1   
412177            9            0    6      6       621           1          1   
472361            2            0    0     24      3328           0          1   
505720            3            0    0     33      2130           1          1

In [67]:
print(imblearn.__version__)

0.7.0


In [68]:
train_df = df_sample

In [69]:
X_df = train_df.drop('record_type',axis=1)

In [70]:
y_recordtype = train_df['record_type']
y_recordtype

16051     0
163505    1
28919     1
180126    1
336471    0
         ..
98737     1
153853    0
412177    0
472361    0
505720    0
Name: record_type, Length: 64565, dtype: int64

In [71]:
# Split the data into a training and test set.
Xlr, Xtestlr, ylr, ytestlr = train_test_split(X_df, 
                                              y_recordtype, test_size=0.1,random_state=42)

In [72]:
Xlr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58108 entries, 516327 to 93429
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   shopping_pt        58108 non-null  int64
 1   day                58108 non-null  int64
 2   state              58108 non-null  int64
 3   location           58108 non-null  int64
 4   group_size         58108 non-null  int64
 5   homeowner          58108 non-null  int64
 6   car_age            58108 non-null  int64
 7   car_value          58108 non-null  int64
 8   risk_factor        58108 non-null  int64
 9   age_oldest         58108 non-null  int64
 10  age_youngest       58108 non-null  int64
 11  married_couple     58108 non-null  int64
 12  c_previous         58108 non-null  int64
 13  duration_previous  58108 non-null  int64
 14  a                  58108 non-null  int64
 15  b                  58108 non-null  int64
 16  c                  58108 non-null  int64
 17  d      

In [73]:
ylr.count()

58108

In [74]:
ylr.value_counts()

0    49476
1     8632
Name: record_type, dtype: int64

In [75]:
oversample = SMOTE()

In [76]:
X, y = oversample.fit_resample(Xlr, ylr)

In [77]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98952 entries, 0 to 98951
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   shopping_pt        98952 non-null  int64
 1   day                98952 non-null  int64
 2   state              98952 non-null  int64
 3   location           98952 non-null  int64
 4   group_size         98952 non-null  int64
 5   homeowner          98952 non-null  int64
 6   car_age            98952 non-null  int64
 7   car_value          98952 non-null  int64
 8   risk_factor        98952 non-null  int64
 9   age_oldest         98952 non-null  int64
 10  age_youngest       98952 non-null  int64
 11  married_couple     98952 non-null  int64
 12  c_previous         98952 non-null  int64
 13  duration_previous  98952 non-null  int64
 14  a                  98952 non-null  int64
 15  b                  98952 non-null  int64
 16  c                  98952 non-null  int64
 17  d           

In [78]:
y.count()

98952

In [79]:
y.value_counts()

0    49476
1    49476
Name: record_type, dtype: int64

After applying SMOTE, number of successful records 1 increased from 57671 to 329717. Now its 50%

In [80]:
# Applyting LogisticRegression

clf = LogisticRegression()
# Fit the model on the trainng data.
clf.fit(X, y)
# Print the accuracy from the testing data.
print(accuracy_score(clf.predict(Xtestlr), ytestlr))

0.7568530277218523


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [81]:
print(ytestlr.value_counts())

0    5504
1     953
Name: record_type, dtype: int64


In [82]:
print(confusion_matrix(ytestlr, clf.predict(Xtestlr)))

[[4190 1314]
 [ 256  697]]


In [83]:
print(classification_report(ytestlr, clf.predict(Xtestlr)))

              precision    recall  f1-score   support

           0       0.94      0.76      0.84      5504
           1       0.35      0.73      0.47       953

    accuracy                           0.76      6457
   macro avg       0.64      0.75      0.66      6457
weighted avg       0.85      0.76      0.79      6457



In [84]:
# Applying KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=6)

In [85]:
knn.fit(X,y)
y_pred_knn = knn.predict(Xtestlr)

In [86]:
print(confusion_matrix(ytestlr, y_pred_knn))

[[3789 1715]
 [ 590  363]]


In [87]:
print(classification_report(ytestlr, y_pred_knn))

              precision    recall  f1-score   support

           0       0.87      0.69      0.77      5504
           1       0.17      0.38      0.24       953

    accuracy                           0.64      6457
   macro avg       0.52      0.53      0.50      6457
weighted avg       0.76      0.64      0.69      6457



Performing hyperprameter tuning, to check with different n_neighbors values

In [88]:

for k in range(1,5):
    print(" Iterating with n_neighbors ", k)
    knn_h = KNeighborsClassifier(n_neighbors=k)
    knn_h.fit(X,y)
    y_pred_knn_h = knn_h.predict(Xtestlr)
    print(confusion_matrix(ytestlr, y_pred_knn_h))
    print(classification_report(ytestlr, y_pred_knn_h))

 Iterating with n_neighbors  1
[[4402 1102]
 [ 666  287]]
              precision    recall  f1-score   support

           0       0.87      0.80      0.83      5504
           1       0.21      0.30      0.25       953

    accuracy                           0.73      6457
   macro avg       0.54      0.55      0.54      6457
weighted avg       0.77      0.73      0.75      6457

 Iterating with n_neighbors  2
[[4652  852]
 [ 729  224]]
              precision    recall  f1-score   support

           0       0.86      0.85      0.85      5504
           1       0.21      0.24      0.22       953

    accuracy                           0.76      6457
   macro avg       0.54      0.54      0.54      6457
weighted avg       0.77      0.76      0.76      6457

 Iterating with n_neighbors  3
[[3845 1659]
 [ 586  367]]
              precision    recall  f1-score   support

           0       0.87      0.70      0.77      5504
           1       0.18      0.39      0.25       953

    accu

In [89]:
# Applying Random Forest

from sklearn.ensemble import RandomForestClassifier

In [90]:
# define the model
rfc = RandomForestClassifier()
# fit the model on the whole dataset
rfc.fit(X, y)

RandomForestClassifier()

In [91]:
yhat = rfc.predict(Xtestlr)

In [92]:
yhat

array([0, 0, 0, ..., 0, 0, 0])

In [93]:
print(confusion_matrix(ytestlr, yhat))

[[4796  708]
 [ 509  444]]


In [94]:
print(classification_report(ytestlr, yhat))

              precision    recall  f1-score   support

           0       0.90      0.87      0.89      5504
           1       0.39      0.47      0.42       953

    accuracy                           0.81      6457
   macro avg       0.64      0.67      0.65      6457
weighted avg       0.83      0.81      0.82      6457



In [95]:
from sklearn.svm import OneClassSVM
from sklearn import svm

In [96]:
#svm_clf = OneClassSVM(kernel='linear',gamma='auto',class_weight='').fit(X)

#svm.SVC(kernel='linear', class_weight={1: 10})

#svm_clf = svm.SVC(kernel='linear', cache_size=7000,C=1.0)
#
#svm_clf =svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)

In [97]:
Xlr.info

<bound method DataFrame.info of         shopping_pt  day  state  location  group_size  homeowner  car_age  \
516327            9    2     26      5998           0          0        3   
621135            3    1     13       289           0          0        3   
261833            1    0      6      2978           0          1        1   
164420            3    3      6       948           0          1       10   
405208            7    3     27      2927           1          1        2   
...             ...  ...    ...       ...         ...        ...      ...   
117240            5    0      2       560           0          1       12   
662571            4    1     23      4621           0          1        4   
167264            2    3      6      4040           0          1        2   
295513            4    0      2      5200           0          0       13   
93429             6    0      3      2771           1          1        3   

        car_value  risk_factor  age_oldest 

In [98]:
ylr.value_counts()

0    49476
1     8632
Name: record_type, dtype: int64

In [99]:
df_sample_red = df_s.sample(frac=0.05, replace=True)

In [100]:
df_sample_red.info

<bound method DataFrame.info of         shopping_pt  record_type  day  state  location  group_size  homeowner  \
247280            7            0    4     32      2221           0          0   
657768            8            1    3     13       541           0          1   
629262            3            0    3      7      5227           0          1   
184450            1            0    3     23      4705           1          0   
54466             3            0    3     20      4438           0          0   
...             ...          ...  ...    ...       ...         ...        ...   
546827            1            0    1      6      1382           0          1   
565984            8            0    2     23      1243           0          1   
637033            4            1    0     23      3152           0          1   
98747             2            0    0     21      3597           0          1   
354677            1            0    0      6      2302           0          1

In [101]:
X_df_red = df_sample_red.drop('record_type',axis=1)
X_df_red

,shopping_pt,day,state,location,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,...,c_previous,duration_previous,a,b,c,d,e,f,g,cost
247280,7,4,32,2221,0,0,10,5,2,22,...,1,6,1,1,1,1,0,2,1,670
657768,8,3,13,541,0,1,4,6,1,32,...,2,0,1,1,2,2,1,1,2,685
629262,3,3,7,5227,0,1,4,5,2,68,...,1,1,0,0,1,1,0,0,1,612
184450,1,3,23,4705,1,0,6,5,2,36,...,0,3,1,1,2,0,1,2,0,749
54466,3,3,20,4438,0,0,1,4,2,19,...,2,1,2,0,2,2,0,3,0,674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546827,1,1,6,1382,0,1,15,3,0,67,...,1,6,1,1,1,2,0,2,2,555
565984,8,2,23,1243,0,1,6,3,0,48,...,2,0,1,0,2,2,0,0,2,595
637033,4,0,23,3152,0,1,1,5,2,56,...,2,6,1,0,2,2,0,0,2,637
98747,2,0,21,3597,0,1,1,3,3,60,...,2,15,1,1,0,0,1,1,0,595


In [102]:
y_recordtype_red = df_sample_red['record_type']
y_recordtype_red

247280    0
657768    1
629262    0
184450    0
54466     0
         ..
546827    0
565984    0
637033    1
98747     0
354677    0
Name: record_type, Length: 32282, dtype: int64

In [103]:
# Split the data into a training and test set.
Xlr_red, Xtestlr_red, ylr_red, ytestlr_red = train_test_split(X_df_red, 
                                              y_recordtype_red, test_size=0.1,random_state=42)

In [104]:
Xlr_red.info

<bound method DataFrame.info of         shopping_pt  day  state  location  group_size  homeowner  car_age  \
18897             2    3     12      1291           0          0       16   
201350            1    1     27      3442           0          0       15   
602978            1    2     12      1062           0          0        7   
494093            3    4      6      3044           1          1        5   
348604            3    1      6       180           0          1       14   
...             ...  ...    ...       ...         ...        ...      ...   
26923             5    2     24      3357           0          1        5   
40682             3    3     28      2544           1          1       10   
229096            3    4      2      2264           0          0       10   
548067            4    4     24      4896           0          0       17   
588707            2    0      6      2698           0          0        2   

        car_value  risk_factor  age_oldest 

In [105]:
ylr_red.count

<bound method Series.count of 18897     0
201350    0
602978    0
494093    0
348604    0
         ..
26923     0
40682     0
229096    0
548067    0
588707    0
Name: record_type, Length: 29053, dtype: int64>

In [106]:
ylr_red.value_counts()

0    24736
1     4317
Name: record_type, dtype: int64

In [107]:
for cw in (0.5,0.8,1.0):
    print("***** Applyting the model with the class weigth - ",cw)
    svm_clf = svm.SVC(kernel='linear', cache_size=7000,C=cw)
    svm_clf.fit(Xlr_red, ylr_red)
    y_svm = svm_clf.predict(Xtestlr_red)
    print("confusion matirx - ",confusion_matrix(ytestlr_red, y_svm))
    print("classification report - ",classification_report(ytestlr_red, y_svm))

***** Applyting the model with the class weigth -  0.5
confusion matirx -  [[2613  134]
 [ 363  119]]
classification report -                precision    recall  f1-score   support

           0       0.88      0.95      0.91      2747
           1       0.47      0.25      0.32       482

    accuracy                           0.85      3229
   macro avg       0.67      0.60      0.62      3229
weighted avg       0.82      0.85      0.83      3229

***** Applyting the model with the class weigth -  0.8
confusion matirx -  [[2604  143]
 [ 360  122]]
classification report -                precision    recall  f1-score   support

           0       0.88      0.95      0.91      2747
           1       0.46      0.25      0.33       482

    accuracy                           0.84      3229
   macro avg       0.67      0.60      0.62      3229
weighted avg       0.82      0.84      0.82      3229

***** Applyting the model with the class weigth -  1.0
confusion matirx -  [[2604  143]
 [ 3